# Retrieve CO-OPS data from an arbitrary time interval and region

In [ ]:
from datetime import datetime

import searvey
import geopandas
import hvplot.pandas
import pandas as pd
import pytz
import shapely
from multifutures import multiprocess
from searvey.coops import get_coops_stations
from searvey._coops_api import fetch_coops_station

# retrieve a [list of CO-OPS stations](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations)

In [ ]:
all_stations = get_coops_stations(metadata_source='main')
all_stations

In [ ]:
world_plot = all_stations.hvplot(geo=True, tiles=True, hover_cols=["nos_id", "location"])
world_plot.opts(width=800, height=500)

# retrieve a [list of CO-OPS stations within an arbitrary polygon](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.coops_stations_within_region)

In [ ]:
# This is the actualBBox of the east coast, but this takes too long to retrieve.
# Since we execute this notebook on CI, let's actually use Florida
#east_coast = shapely.geometry.box(-85, 25, -65, 45)
east_coast = shapely.geometry.box(-85, 25, -65, 30)
east_coast_stations = get_coops_stations(metadata_source='main', region=east_coast)
east_coast_stations
east_coast_stations.hvplot(geo=True, tiles=True, hover_cols=["nos_id", "location"])

In [ ]:
east_coast_stations.hvplot(geo=True, tiles=True, by='status', legend=True, title='CO-OPS stations on the U.S. East Coast')

# retrieve a CO-OPS data product from a [specific station](https://searvey.readthedocs.io/en/latest/coops.html#searvey.coops.COOPS_Station)

In [ ]:
station_id = '1612480'
station_water_levels = fetch_coops_station(
    station_id=station_id,
    start_date=datetime(2022, 4, 2, 12, tzinfo=pytz.utc),
    end_date=datetime(2022, 4, 2, 12, 30, tzinfo=pytz.utc),
    product='water_level',
)
station_water_levels

In [ ]:
timeseries_plot = (
    station_water_levels.hvplot(x='time', y='value')
    * station_water_levels.hvplot.errorbars(x='time', y='value', yerr1='sigma')
)
start_date = pd.to_datetime(station_water_levels.index.min())
end_date = pd.to_datetime(station_water_levels.index.max())
timeseries_plot.opts(title=f'water level observed by CO-OPS station {station_id} between {start_date} and {end_date}')

# retrieve a CO-OPS data product from a lsit of stations

In [ ]:
stations = east_coast_stations[(east_coast_stations.status == 'active') & (east_coast_stations.station_type == 'waterlevels')]
station_water_levels = pd.concat(
    [
        result.result for result in multiprocess(
            fetch_coops_station,
            [
                {
                    'station_id': nos_id,
                    'start_date': datetime(2022, 4, 2, 12, tzinfo=pytz.utc),
                    'end_date': datetime(2022, 4, 2, 12, 30, tzinfo=pytz.utc),
                    'product': 'water_level',
                } for nos_id, _ in stations.iterrows()
            ],
        ) 
    ],
    keys=[nos_id for nos_id, _ in stations.iterrows()],
    names=['nos_id', 'time']
)
station_water_levels

# additional documentation
## https://searvey.readthedocs.io
## https://github.com/oceanmodeling/searvey
